In [16]:
import os
import tqdm
import sys

In [28]:
PASSAGES_CHUNK=5e+6

CORPUS_CHUNK_FILENAME_FORMAT="clueweb22-pt/{}_{:03}.tsv"

In [8]:
corpus_filename = "clueweb22-pt/clueweb22_pt_colbertx_01.tsv"
corpus_map_filename = "clueweb22-pt/clueweb22_pt_colbertx_to_clueweb_id_01.tsv"

corpus_chunks = 0

In [11]:
total_passages = !wc -l {corpus_map_filename}

### Find out where to split the corpus avoiding separating same document passages

In [25]:
chunk_init = [0]

with open(corpus_map_filename) as input_file:
    
    previous_original_doc_id = ""
    
    ready_to_save = False
    
    for i, line in enumerate(tqdm.tqdm(input_file, total=int(total_passages[0].split(" ")[0]), file=sys.stdout, position=0, leave=True)):
        colbertx_id, original_id = line.split('\t')
        
        original_doc_id = original_id.split("__")[0]
        
        if not ready_to_save:
            if ((i + 1) % PASSAGES_CHUNK == 0):
                ready_to_save = True
                print(line.strip())
        else:
            if original_doc_id != previous_original_doc_id:
                chunk_init.append(i)
                
                ready_to_save = False
                
            print(line.strip())
                
        previous_original_doc_id = original_doc_id

 22%|██▏       | 4947368/22764702 [00:03<00:14, 1193422.09it/s]4999999	clueweb22-pt0000-21-15791__26
5000000	clueweb22-pt0000-21-15791__28
5000001	clueweb22-pt0000-21-15791__30
5000002	clueweb22-pt0000-21-15791__32
5000003	clueweb22-pt0000-21-15791__34
5000004	clueweb22-pt0000-21-15791__36
5000005	clueweb22-pt0000-21-15791__38
5000006	clueweb22-pt0000-21-15791__40
5000007	clueweb22-pt0000-21-15791__42
5000008	clueweb22-pt0000-21-15791__44
5000009	clueweb22-pt0000-21-15791__46
5000010	clueweb22-pt0000-21-15791__48
5000011	clueweb22-pt0000-21-15791__50
5000012	clueweb22-pt0000-21-15791__52
5000013	clueweb22-pt0000-21-15791__54
5000014	clueweb22-pt0000-21-15791__56
5000015	clueweb22-pt0000-21-15792__0
 44%|████▎     | 9950069/22764702 [00:07<00:09, 1349312.17it/s]9999999	clueweb22-pt0000-44-03497__36
10000000	clueweb22-pt0000-44-03497__38
10000001	clueweb22-pt0000-44-03497__40
10000002	clueweb22-pt0000-44-03497__42
10000003	clueweb22-pt0000-44-03497__44
10000004	clueweb22-pt0000-44-03497_

In [22]:
chunk_init

[0, 5000015, 10000015, 15000009, 20000002]

### Now split the corpus

In [33]:
with open(corpus_filename) as input_file:
    output_file = None
    output_files_counter = 0
    
    output_passages_counter = 0
    output_file = open(CORPUS_CHUNK_FILENAME_FORMAT.format(os.path.splitext(os.path.basename(corpus_filename))[0], output_files_counter), "w")
    
    for i, line in enumerate(tqdm.tqdm(input_file, total=int(total_passages[0].split(" ")[0]), file=sys.stdout, position=0, leave=True)):
        if output_files_counter < len(chunk_init[1:]) and i == chunk_init[1:][output_files_counter]:
            output_file.close()
            
            output_files_counter += 1
            
            output_passages_counter = 0
            output_file = open(CORPUS_CHUNK_FILENAME_FORMAT.format(os.path.splitext(os.path.basename(corpus_filename))[0], output_files_counter), "w")
                
        if output_passages_counter > 0:
            output_file.write("\n")
            
        output_file.write(line.strip())
        
        output_passages_counter += 1
        
    output_file.close()

22764703it [03:22, 112620.35it/s]                              
